# 2024-12-06

## Data

In [65]:
arr = []
open("../inputs/2024-12-06.txt", "r") do file
    for line in eachline(file)
        chars = split(line, r"")
        push!(arr, String.(chars))
	end
end

n = length(arr)
m = length(arr[1])
mat = Matrix{String}(undef, n,m)

for i in 1:n
    for j in 1:m
        mat[i,j] = arr[i][j]
    end
end

## Part I

In [66]:
mat[ [initial_position[1], initial_position[2]] + [1,0] ]

2-element Vector{String}:
 "."
 "."

In [67]:
pos = findfirst(x -> x in ["^", "v", "<", ">"], mat)
dir = Dict(
    "^" => [-1, 0],
    "v" => [1, 0],
    "<" => [0, -1],
    ">" => [0, 1],
)[mat[pos]]
dir = CartesianIndex(dir...)
R = [
    0 1;
    -1 0
]
left_area = false
while !left_area
    ray = [pos + j * dir for j in 1:n if checkbounds(Bool, mat, pos + j * dir)]
    k = findfirst(x -> x == "#", mat[ray])
    if k === nothing
        left_area = true
        mat[ray] .= "X"
        continue
    end
    pos = ray[k - 1]
    dir = CartesianIndex((R * [dir[1], dir[2]])...)
    mat[ray[1:(k-1)]] .= "X"
end

In [ ]:
count(x -> x == "X", mat) 

4753

## Part II